In [1]:
import pandas as pd
import geopandas as gpd

# Cargar Datos

In [8]:
# importar capas
gdf_evp = gpd.read_file("./CapasEditadas/EspaciosPublicos_v6.geojson")
gdf_evp.drop(columns=['barp_nam', 'barp_bool', 'barp100_cant', 'barp100_nam'], inplace=True)

In [9]:
# importar capas de municipios y límite antrópico
gdf_muni = gpd.read_file("../1_Datos/ACUMAR/LIMITE_POLITICO.shp")
gdf_limite = gpd.read_file("../1_Datos/ACUMAR/LIMITE_ANTROPICO_POLIGONO.geojson")

In [10]:
# importar barrios populares
gdf_barrios = gpd.read_file("../1_Datos/Renabap/BarriosPopulares_2022-05-15_info_publica.geojson")
gdf_barrios.head()

,renabap_id,nombre_barrio,provincia,departamento,localidad,cantidad_familias_aproximada,cantidad_viviendas_aproximadas,decada_de_creacion,anio_de_creacion,energia_electrica,efluentes_cloacales,agua_corriente,cocina,calefaccion,situacion_dominial,clasificacion_barrio,superficie_m2,geometry
0,1,Monterrey I,Buenos Aires,Pilar,Presidente Derqui,44.0,40.0,Década 1990,NaN,Conexión a la red con medidor comunitario,Desagüe a cámara séptica y pozo ciego,Bomba de agua de pozo domiciliaria,Gas en garrafa,Otro / vacío,Otro tipo de seguridad en la tenencia,Asentamiento,11674,"MULTIPOLYGON (((-58.83350 -34.48128, -58.83383..."
1,2,Malvinas II,Buenos Aires,La Plata,José Melchor Romero,319.0,290.0,Década 1990,NaN,Conexión irregular a la red,Desagüe a cámara séptica y pozo ciego,Conexión irregular a la red de agua,Gas en garrafa,Otro / vacío,Ninguna seguridad en la tenencia,Asentamiento,98034,"MULTIPOLYGON (((-58.01200 -34.94732, -58.00911..."
2,3,Ferroviario,Buenos Aires,La Plata,Angel Etcheverry,146.0,133.0,Década 2000,NaN,Conexión irregular a la red,Desagüe sólo a pozo negro/ciego u hoyo,Conexión formal a la red de agua con factura,Gas en garrafa,Leña o carbón,Ninguna seguridad en la tenencia,Asentamiento,75850,"MULTIPOLYGON (((-58.07980 -35.03507, -58.07958..."
3,4,La Favelita,Buenos Aires,La Plata,Tolosa,134.0,122.0,Década 1970,NaN,Conexión irregular a la red,Desagüe sólo a pozo negro/ciego u hoyo,Conexión irregular a la red de agua,Gas en garrafa,Energía eléctrica,Ninguna seguridad en la tenencia,Asentamiento,36865,"MULTIPOLYGON (((-57.97991 -34.90834, -57.97994..."
4,5,Casaca,Buenos Aires,La Plata,City Bell,22.0,20.0,Década 2000,NaN,Conexión irregular a la red,Desagüe a cámara séptica y pozo ciego,Bomba de agua de pozo domiciliaria,Gas en garrafa,Otro / vacío,Ninguna seguridad en la tenencia,Asentamiento,25754,"MULTIPOLYGON (((-58.06297 -34.90134, -58.06328..."


# Eliminar polígonos duplicados

In [11]:
for i in gdf_evp.loc[gdf_evp.fuente == "UNSAM"].index:
    
    gdf_join = gpd.sjoin(gdf_evp.loc[[i]], gdf_evp.loc[(gdf_evp.fuente == "UNSAM")&(gdf_evp.index != i)], how='left')
    if len(gdf_join.loc[gdf_join.index_right.notnull()]) >0:
        print (gdf_join.loc[gdf_join.index_right.notnull(), ['full_id_left', 'full_id_right']])

In [12]:
""" print(len(gdf_evp))
gdf_evp = gdf_evp.loc[gdf_evp.full_id != 'usm00002']
gdf_evp = gdf_evp.loc[gdf_evp.full_id != 'usm00004']
gdf_evp = gdf_evp.loc[gdf_evp.full_id != 'usm00005']
gdf_evp.reset_index(inplace=True, drop=True)
print(len(gdf_evp)) """

" print(len(gdf_evp))\ngdf_evp = gdf_evp.loc[gdf_evp.full_id != 'usm00002']\ngdf_evp = gdf_evp.loc[gdf_evp.full_id != 'usm00004']\ngdf_evp = gdf_evp.loc[gdf_evp.full_id != 'usm00005']\ngdf_evp.reset_index(inplace=True, drop=True)\nprint(len(gdf_evp)) "

# Calcular superficie

In [3]:
gdf_evp = gdf_evp.to_crs("EPSG:5347")
gdf_evp['area_m2'] = gdf_evp.area
gdf_evp.area_m2 = round(gdf_evp.area_m2, 2)
gdf_evp = gdf_evp.to_crs("EPSG:4326")

# Crear nuevo campo bool y nombre de barrios popular

In [14]:
# presencia y nombre de barrio al que pertenece el espacio verde
gdf_evp_b = gpd.sjoin(gdf_evp, gdf_barrios, how='left')
gdf_evp_b.drop_duplicates(subset=['full_id'], inplace=True)
gdf_evp = gdf_evp.merge(gdf_evp_b[['full_id', 'nombre_barrio']], on='full_id', how='left')
gdf_evp.rename(columns={'nombre_barrio':'barp_nam'}, inplace=True)
gdf_evp['barp_bool'] = 0
gdf_evp.loc[gdf_evp.barp_nam.notnull(), 'barp_bool'] = 1

In [15]:
# buffer de 100m y cruce de capas
gdf_evp_buffer = gdf_evp.copy()
gdf_evp_buffer = gdf_evp_buffer.to_crs("EPSG:5347")
gdf_evp_buffer.geometry = gdf_evp_buffer.buffer(100)
gdf_evp_buffer = gdf_evp_buffer.to_crs("EPSG:4326")

gdf_evp_buffer_b = gpd.sjoin(gdf_evp_buffer, gdf_barrios, how='left')

In [16]:
# cantidad y nombres de barrios a 100 de cada espacio verde
pd_bar100_size = pd.DataFrame(gdf_evp_buffer_b.loc[gdf_evp_buffer_b.nombre_barrio.notnull(), 'full_id'].value_counts())
pd_bar100_size.reset_index(inplace=True)
pd_bar100_size.rename(columns={'index':'full_id', 'full_id':'barp100_cant'}, inplace=True)
pd_bar100_nam = pd.DataFrame(gdf_evp_buffer_b.loc[gdf_evp_buffer_b.nombre_barrio.notnull()].groupby(['full_id'])['nombre_barrio'].apply(lambda x: (", ").join(list(set(x)))))
pd_bar100_nam.reset_index(inplace=True)

In [17]:
gdf_evp = gdf_evp.merge(pd_bar100_size[['full_id', 'barp100_cant']], on='full_id', how='left')
gdf_evp = gdf_evp.merge(pd_bar100_nam[['full_id', 'nombre_barrio']], on='full_id', how='left')
gdf_evp.rename(columns={'nombre_barrio':'barp100_nam'}, inplace=True)
gdf_evp.barp100_cant.fillna(0, inplace=True)

# Generar valores de campo "full_id" para los datos UNSAM 

In [18]:
num = 1
for i in gdf_evp.loc[gdf_evp.fuente == "UNSAM"].index:
    num_str = str(num).zfill(5)
    gdf_evp.loc[i, 'full_id'] = "usm" + num_str
    num = num+1

# Limpieza de datos

In [19]:
# check layer size
len(gdf_evp)

301

In [20]:
# check duplicated polygons 
for i in gdf_evp.index:
    
    gdf_join = gpd.sjoin(gdf_evp.loc[[i]], gdf_evp.loc[(gdf_evp.index != i)], how='left')
    if len(gdf_join.loc[gdf_join.index_right.notnull()]) >0:
        print (gdf_join.loc[gdf_join.index_right.notnull(), ['full_id_left', 'full_id_right']])

   full_id_left full_id_right
11   w291374429    w302130495
   full_id_left full_id_right
15   w302130495    w291374429
    full_id_left full_id_right
121   w224939240    w224939242
    full_id_left full_id_right
122   w224939242    w224939240
    full_id_left full_id_right
173   w224939237    w224939239
    full_id_left full_id_right
174   w224939239    w224939237
    full_id_left full_id_right
203    w28184879    w167029793
    full_id_left full_id_right
206   w167029793     w28184879


In [21]:
# check polígonos eliminados
lista_e = ['w149700227', 'r5514857', 'w241322368', 'r2895800', '5902', '13221', '13562', 'w678895041', '107', 'w557932545', 'w557932547', 'w557932548',
        'r2467321', 'w160069211', 'w160069212', 'w160069213', 'w167029792', 'w160069208', 'w800490320', 'w160069206', 'w570792577', 'w510623913']

for e in lista_e:
    if len(gdf_evp.loc[gdf_evp.full_id == i]) > 0:
        print(i)

In [22]:
# check unique ids
len(gdf_evp) == len(gdf_evp.full_id.unique())

True

In [23]:
# check unique tipo
gdf_evp.tipo.unique()

array(['Plaza', 'Plazoleta', 'Parque con equipamiento', '-', 'Parque',
       'Separador vial', 'Plazoleta/Separador vial', '?', 'Cancha', None],
      dtype=object)

In [24]:
gdf_evp.loc[gdf_evp.tipo == 'Separador Vial', 'tipo'] = 'Separador vial'

In [25]:
# check unique distrito
gdf_evp.distrito.unique()

array(['Lomas de Zamora', 'Lanús'], dtype=object)

In [26]:
gdf_evp.loc[gdf_evp.distrito == 'Lanus', 'distrito'] = 'Lanús'

In [27]:
# recalcular limite
gdf_evp = gpd.sjoin(gdf_evp, gdf_limite[['fid', 'geometry']], how='left')
gdf_evp['limite'] = 1
gdf_evp.loc[gdf_evp.index_right.isnull(), 'limite'] = 0

In [28]:
# ordenar columns y eliminar "superpuesto"
gdf_evp = gdf_evp[['full_id', 'fuente', 'distrito', 'evp', 'tipo', 'nombre', 'sis_bool', 'sis_nam',
        'area_m2', 'barp_nam', 'barp_bool', 'barp100_cant', 'barp100_nam', 'limite', 'observaciones', 
        'geometry']]

In [29]:
gdf_evp.to_file("./CapasEditadas/EspaciosPublicos_v6.1.geojson", driver="GeoJSON")